In [1]:
import pandas as pd
from abc import ABC, abstractmethod
import nbimporter
from train import get_cryptoquant_data

In [2]:
class IIndicator(ABC):
    @abstractmethod
    def indicator(self, cryptoquant_data):
        pass

#### RSI

In [3]:
class RSIIndicator(IIndicator):
    def __init__(self, window=14):
        self.window = window

    def indicator(self, cryptoquant_data):

        # Calculate RSI
        delta = cryptoquant_data["close"].diff()
        gain = delta.where(delta > 0, 0).rolling(window=self.window).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=self.window).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))

        cryptoquant_data["rsi"] = rsi

        return cryptoquant_data

#### Momentum

In [4]:
class MomentumIndicator(IIndicator):
    def __init__(self, short_window=10, long_window=50):
        self.short_window = short_window
        self.long_window = long_window

    def indicator(self, cryptoquant_data):

        # Calculate Momentum
        cryptoquant_data["short_ma"] = cryptoquant_data["close"].rolling(self.short_window).mean()
        cryptoquant_data["long_ma"] = cryptoquant_data["close"].rolling(self.long_window).mean()

        return cryptoquant_data


#### Bollinger Bands

In [5]:
class BollingerBands(IIndicator):
    def __init__(self, window=20, num_std=2):
        self.window = window
        self.num_std = num_std

    def indicator(self, cryptoquant_data):

        # Calculate Bollinger Bands
        cryptoquant_data["MA"] = cryptoquant_data["close"].rolling(window=self.window).mean()
        cryptoquant_data["STD"] = cryptoquant_data["close"].rolling(window=self.window).std()
        cryptoquant_data["Upper_2"] = cryptoquant_data["MA"] + (
            self.num_std * cryptoquant_data["STD"]
        )
        cryptoquant_data["Lower_2"] = cryptoquant_data["MA"] - (
            self.num_std * cryptoquant_data["STD"]
        )
        cryptoquant_data["Upper_1"] = cryptoquant_data["MA"] + cryptoquant_data["STD"]
        cryptoquant_data["Lower_1"] = cryptoquant_data["MA"] - cryptoquant_data["STD"]

        return cryptoquant_data

#### Stochastic Oscillator

In [6]:
class StochasticOscillator(IIndicator):
    def __init__(self, window = 20):
        self.window = window

    def indicator(self, cryptoquant_data):

        # Calculate Stochastic Oscillator
        low_min = cryptoquant_data["low"].rolling(window=self.window).min()
        high_max = cryptoquant_data["high"].rolling(window=self.window).max()
        k = 100 * (cryptoquant_data["close"] - low_min) / (high_max - low_min)
        d = k.rolling(window=3).mean()

        cryptoquant_data["low_min"] = low_min
        cryptoquant_data["high_max"] = high_max
        cryptoquant_data["sto_k"] = k
        cryptoquant_data["sto_d"] = d

        return cryptoquant_data

#### MACD

In [7]:
class MACDIndicator(IIndicator):
    def indicator(self, cryptoquant_data):
        
        # Calculate MACD
        exp1 = cryptoquant_data["close"].ewm(span=12, adjust=False).mean()
        exp2 = cryptoquant_data["close"].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()

        cryptoquant_data["exp1"] = exp1
        cryptoquant_data["exp2"] = exp2
        cryptoquant_data["macd"] = macd
        cryptoquant_data["macd_signal"] = signal

        return cryptoquant_data

In [8]:
def get_indicator_data():
    data = get_cryptoquant_data()
    data = RSIIndicator().indicator(data)
    data = MomentumIndicator().indicator(data)
    data = BollingerBands().indicator(data)
    data = StochasticOscillator().indicator(data)
    data = MACDIndicator().indicator(data)
    
    return data

In [9]:
indicator_data = get_indicator_data()
print(indicator_data.head(100))

                        close      high       low      open        volume  \
datetime                                                                    
2019-04-12 13:00:00  0.325668  0.329023  0.324243  0.327329  3.992486e+06   
2019-04-12 14:00:00  0.326387  0.328087  0.324068  0.325144  3.126063e+06   
2019-04-12 15:00:00  0.325266  0.327929  0.324561  0.326745  3.879088e+06   
2019-04-12 16:00:00  0.327117  0.329773  0.325294  0.325046  3.774059e+06   
2019-04-12 17:00:00  0.326224  0.328552  0.325843  0.326846  3.524562e+06   
...                       ...       ...       ...       ...           ...   
2019-04-16 12:00:00  0.318343  0.320678  0.317775  0.319355  3.009291e+06   
2019-04-16 13:00:00  0.318317  0.319465  0.317294  0.317875  2.022290e+06   
2019-04-16 14:00:00  0.319592  0.319592  0.316051  0.318281  4.001572e+06   
2019-04-16 15:00:00  0.318906  0.321205  0.318403  0.319014  2.460008e+06   
2019-04-16 16:00:00  0.318865  0.320563  0.318326  0.318975  1.856081e+06   